In [2]:
import pandas as pd
from pycoingecko import CoinGeckoAPI

# Initialiser le client CoinGeckoAPI
cg = CoinGeckoAPI()

# Paramètres pour la requête API
parameters = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': 50,
    'page': 1,
    'sparkline': False,
    'locale': 'en'
}

# Récupérer les données du marché
coin_market_data = cg.get_coins_markets(**parameters)

# Convertir les données en DataFrame
df = pd.DataFrame(coin_market_data)

# Sélectionner les IDs des cryptomonnaies
top_50_coins = df[['id']].head(50)



In [3]:
import pandas as pd
from pycoingecko import CoinGeckoAPI
import os

# Initialiser le client CoinGeckoAPI
cg = CoinGeckoAPI()

# Fonction pour récupérer les données historiques d'une cryptomonnaie
def fetch_historical_data(coin_id, start_date, end_date):
    start_timestamp = int(pd.Timestamp(start_date).timestamp())
    end_timestamp = int(pd.Timestamp(end_date).timestamp())
    historical_data = cg.get_coin_market_chart_range_by_id(id=coin_id, vs_currency='usd',
                                                           from_timestamp=start_timestamp, to_timestamp=end_timestamp)
    prices = historical_data['prices']
    df_historical = pd.DataFrame(prices, columns=['timestamp', 'price'])
    df_historical['timestamp'] = pd.to_datetime(df_historical['timestamp'], unit='ms')
    df_historical.set_index('timestamp', inplace=True)
    return df_historical

# Récupérer les données du marché pour le Top 50 des cryptomonnaies
market_data = cg.get_coins_markets(vs_currency='usd', order='market_cap_desc', per_page=50, page=1)
df_market = pd.DataFrame(market_data)
top_50_coins = df_market.head(50)

# Boucle pour récupérer les données historiques pour chaque cryptomonnaie
historical_dataframes = {}
for index, row in top_50_coins.iterrows():
    coin_id = row['id']
    print(f"Téléchargement des données pour {coin_id}.")
    df_historical = fetch_historical_data(coin_id, '2023-01-01', '2024-03-08')
    historical_dataframes[coin_id] = df_historical

# Créer un dossier pour stocker les fichiers CSV
data_dir = 'historical_data'
os.makedirs(data_dir, exist_ok=True)

# Enregistrer chaque DataFrame dans un fichier CSV
for coin_id, df_historical in historical_dataframes.items():
    file_path = os.path.join(data_dir, f"{coin_id}_historical_data.csv")
    df_historical.to_csv(file_path)

print("Les données historiques ont été téléchargées et enregistrées avec succès.")

Téléchargement des données pour bitcoin.
Téléchargement des données pour ethereum.
Téléchargement des données pour tether.
Téléchargement des données pour binancecoin.
